In [56]:
import torch
from torch import nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import platform

if platform.system() == 'Linux':
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


#### The objective here is to one hot encode the words 

so for that we will asing a number to each character and then one hot encode that info in a array of the length of the num of characters

In [ ]:
if platform.system() == "Linux":
    with open ('/content/drive/MyDrive/deep learning/courses/pytorch_course//PYTORCH_NOTEBOOKS/PYTORCH_NOTEBOOKS/Data/shakespeare.txt', 'r', encoding='utf8') as f:
        text = f.read()
else:
    with open ('../../PYTORCH_NOTEBOOKS/PYTORCH_NOTEBOOKS/Data/shakespeare.txt', 'r', encoding='utf8') as f:
        text = f.read()

In [ ]:
print(type(text))
print(len(text))

<class 'str'>
5445609


In [ ]:
print(text[:1000])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world's due, by the grave and thee.


                     2
  When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tattered weed of small worth held:  
  Then being asked, where all thy beauty lies,
  Where all the treasure of thy lusty days;
  To say within thine own deep su

In [ ]:
all_characters = set(text)

print(all_characters, len(all_characters))

{'b', 'O', 'p', '}', '2', '?', '5', 'v', 'E', '`', 'A', '[', 'g', '(', 'j', '7', 'i', 'f', 'T', 'I', 'C', '|', 't', '1', 'k', '\n', 'J', 's', '>', '-', 'L', '8', 'W', 'x', 'S', 'c', 'M', '"', ':', 'P', '9', 'Q', 'N', '4', 'V', 'B', '_', 'e', 'F', 'z', 'Y', 'h', 'G', '3', 'X', 'l', 'Z', '6', ';', 'm', 'a', '0', 'q', 'u', 'd', 'n', 'o', "'", '<', 'D', 'y', '!', 'K', ']', 'w', 'R', ' ', 'U', '.', 'r', ')', ',', '&', 'H'} 84


In [ ]:
for pair in enumerate(all_characters):
    print(pair)


(0, 'b')
(1, 'O')
(2, 'p')
(3, '}')
(4, '2')
(5, '?')
(6, '5')
(7, 'v')
(8, 'E')
(9, '`')
(10, 'A')
(11, '[')
(12, 'g')
(13, '(')
(14, 'j')
(15, '7')
(16, 'i')
(17, 'f')
(18, 'T')
(19, 'I')
(20, 'C')
(21, '|')
(22, 't')
(23, '1')
(24, 'k')
(25, '\n')
(26, 'J')
(27, 's')
(28, '>')
(29, '-')
(30, 'L')
(31, '8')
(32, 'W')
(33, 'x')
(34, 'S')
(35, 'c')
(36, 'M')
(37, '"')
(38, ':')
(39, 'P')
(40, '9')
(41, 'Q')
(42, 'N')
(43, '4')
(44, 'V')
(45, 'B')
(46, '_')
(47, 'e')
(48, 'F')
(49, 'z')
(50, 'Y')
(51, 'h')
(52, 'G')
(53, '3')
(54, 'X')
(55, 'l')
(56, 'Z')
(57, '6')
(58, ';')
(59, 'm')
(60, 'a')
(61, '0')
(62, 'q')
(63, 'u')
(64, 'd')
(65, 'n')
(66, 'o')
(67, "'")
(68, '<')
(69, 'D')
(70, 'y')
(71, '!')
(72, 'K')
(73, ']')
(74, 'w')
(75, 'R')
(76, ' ')
(77, 'U')
(78, '.')
(79, 'r')
(80, ')')
(81, ',')
(82, '&')
(83, 'H')


#### The decoder transform from a num --> letter

In [ ]:
decoder = dict(enumerate(all_characters))

### The encoder take a letter --> num

In [ ]:
decoder.items()

dict_items([(0, 'b'), (1, 'O'), (2, 'p'), (3, '}'), (4, '2'), (5, '?'), (6, '5'), (7, 'v'), (8, 'E'), (9, '`'), (10, 'A'), (11, '['), (12, 'g'), (13, '('), (14, 'j'), (15, '7'), (16, 'i'), (17, 'f'), (18, 'T'), (19, 'I'), (20, 'C'), (21, '|'), (22, 't'), (23, '1'), (24, 'k'), (25, '\n'), (26, 'J'), (27, 's'), (28, '>'), (29, '-'), (30, 'L'), (31, '8'), (32, 'W'), (33, 'x'), (34, 'S'), (35, 'c'), (36, 'M'), (37, '"'), (38, ':'), (39, 'P'), (40, '9'), (41, 'Q'), (42, 'N'), (43, '4'), (44, 'V'), (45, 'B'), (46, '_'), (47, 'e'), (48, 'F'), (49, 'z'), (50, 'Y'), (51, 'h'), (52, 'G'), (53, '3'), (54, 'X'), (55, 'l'), (56, 'Z'), (57, '6'), (58, ';'), (59, 'm'), (60, 'a'), (61, '0'), (62, 'q'), (63, 'u'), (64, 'd'), (65, 'n'), (66, 'o'), (67, "'"), (68, '<'), (69, 'D'), (70, 'y'), (71, '!'), (72, 'K'), (73, ']'), (74, 'w'), (75, 'R'), (76, ' '), (77, 'U'), (78, '.'), (79, 'r'), (80, ')'), (81, ','), (82, '&'), (83, 'H')])

In [ ]:
encoder = {char: ind for ind, char in decoder.items()}

In [ ]:
encoder

{'\n': 25,
 ' ': 76,
 '!': 71,
 '"': 37,
 '&': 82,
 "'": 67,
 '(': 13,
 ')': 80,
 ',': 81,
 '-': 29,
 '.': 78,
 '0': 61,
 '1': 23,
 '2': 4,
 '3': 53,
 '4': 43,
 '5': 6,
 '6': 57,
 '7': 15,
 '8': 31,
 '9': 40,
 ':': 38,
 ';': 58,
 '<': 68,
 '>': 28,
 '?': 5,
 'A': 10,
 'B': 45,
 'C': 20,
 'D': 69,
 'E': 8,
 'F': 48,
 'G': 52,
 'H': 83,
 'I': 19,
 'J': 26,
 'K': 72,
 'L': 30,
 'M': 36,
 'N': 42,
 'O': 1,
 'P': 39,
 'Q': 41,
 'R': 75,
 'S': 34,
 'T': 18,
 'U': 77,
 'V': 44,
 'W': 32,
 'X': 54,
 'Y': 50,
 'Z': 56,
 '[': 11,
 ']': 73,
 '_': 46,
 '`': 9,
 'a': 60,
 'b': 0,
 'c': 35,
 'd': 64,
 'e': 47,
 'f': 17,
 'g': 12,
 'h': 51,
 'i': 16,
 'j': 14,
 'k': 24,
 'l': 55,
 'm': 59,
 'n': 65,
 'o': 66,
 'p': 2,
 'q': 62,
 'r': 79,
 's': 27,
 't': 22,
 'u': 63,
 'v': 7,
 'w': 74,
 'x': 33,
 'y': 70,
 'z': 49,
 '|': 21,
 '}': 3}

In [ ]:
value = encoder['a']
print(value)
print(decoder[value])

60
a


In [ ]:
encoded_text = np.array([encoder[char] for char in text])

In [ ]:
def one_hot_encoder(encoded_text, num_char):
    encoded = np.zeros((encoded_text.size, num_char))
    encoded = encoded.astype(np.float32)
    encoded[np.arange(encoded.shape[0]), encoded_text.flatten()] = 1.0
    encoded = encoded.reshape((*encoded_text.shape, num_char))

    return encoded

In [ ]:
one_hot_encoder(np.array([0, 2, 1]), 3)

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)

### Now we are going to create all the batches

This batches it consist basically that the words are shifted one for example if we have Hello how are you as the text we could do this

h e l l o h o

e l l o h o w

And then we will continue doind this through the dataset

X --> Encoded text of length seq_len

Y --> encoded text shifted by one

Plotting this shit basically the *samp_per_batch* is how many batches do you want and the *seq_len* is how many values do you have inside of each batch

In [ ]:
def generate_batches(encoded_text, samp_per_batch=10, seq_len=50):

    # the num of characters that are in each batch
    char_per_batch = samp_per_batch * seq_len

    # The num of abailable batches with the dataset
    num_batches_avail = int(len(encoded_text)/char_per_batch)

    # drop out the values that don't fit inside of the batches
    encoded_text = encoded_text[:num_batches_avail * char_per_batch]

    encoded_text = encoded_text.reshape((samp_per_batch, -1))
    for n in range(0, encoded_text.shape[1], seq_len):
        x = encoded_text[:, n:n+seq_len]
        y = np.zeros_like(x)

        try:
            y[:, :-1] = x[:, 1:]
            y[:, -1]  = encoded_text[:, n+seq_len]

        except:
            y[:, :-1] = x[:, 1:]
            y[:, -1] = encoded_text[:, 0]

        yield x, y

In [ ]:
sample_text = encoded_text[:20]
sample_text

array([25, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76,
       76, 76, 76])

In [ ]:
batch_generator = generate_batches(sample_text, 2, 5)

In [ ]:
x, y = next(batch_generator)

In [ ]:
x

array([[25, 76, 76, 76, 76],
       [76, 76, 76, 76, 76]])

In [ ]:
y

array([[76, 76, 76, 76, 76],
       [76, 76, 76, 76, 76]])

In [ ]:
sample_nums = np.arange(20)

In [ ]:
batch_generator = generate_batches(sample_nums, samp_per_batch=2, seq_len=5)

In [ ]:
for x, y in batch_generator:
    print(f'x: {x}')
    print()
    print(f'y: {y}')
    print()

x: [[ 0  1  2  3  4]
 [10 11 12 13 14]]

y: [[ 1  2  3  4  5]
 [11 12 13 14 15]]

x: [[ 5  6  7  8  9]
 [15 16 17 18 19]]

y: [[ 6  7  8  9  0]
 [16 17 18 19 10]]



In [ ]:
y

array([[ 6,  7,  8,  9,  0],
       [16, 17, 18, 19, 10]])

In [ ]:
y

array([[ 6,  7,  8,  9,  0],
       [16, 17, 18, 19, 10]])

### Define the LSTM model
The hidden size it corresponds to these three values

(**Number of layers**,                           **Batch size**,                  **Hidden size**)

the number of layers of the lstm model | the size of the batches | the number of neurons that we have

num_layers | batch_size | num_hidden

In [57]:
class CharModel(nn.Module):
    def __init__(self, all_chars, num_hidden=256, num_layers=4, drop_prob=0.4, use_gpu=False):
        super().__init__()
        self.drop_prob = drop_prob
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        self.use_gpu = use_gpu

        self.all_chars = all_chars
        self.decoder = dict(enumerate(self.all_chars))
        self.encoder = {char:ind for ind, char in decoder.items()}

        self.lstm = nn.LSTM(len(self.all_chars), num_hidden, num_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(self.drop_prob)
        self.fc_linear = nn.Linear(num_hidden, len(self.all_chars))

    def forward(self, x, hidden):
        lstm_output, hidden = self.lstm(x, hidden)
        drop_output = self.dropout(lstm_output)
        drop_output = drop_output.contiguous().view(-1, self.num_hidden)
        final_out = self.fc_linear(drop_output)
        return final_out, hidden

    def hidden_state(self, batch_size):
        if self.use_gpu:
            hidden = (torch.zeros( self.num_layers, batch_size, self.num_hidden).cuda(),
                      torch.zeros( self.num_layers, batch_size, self.num_hidden).cuda())
        else:
            hidden = (torch.zeros( self.num_layers, batch_size, self.num_hidden),
                      torch.zeros( self.num_layers, batch_size, self.num_hidden))
        return hidden


In [ ]:
if torch.cuda.is_available():
    use_gpu = True
else:
    use_gpu = False

In [ ]:
model = CharModel(
    all_chars=all_characters,
    num_hidden=312,
    num_layers=3,
    drop_prob=0.5,
    use_gpu=use_gpu,
)

In [ ]:
total_param = []
for p in model.parameters():
    total_param.append(int(p.numel()))

sum(total_param)

2085492

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
train_percent = 0.1

In [ ]:
train_ind = int(len(encoded_text) * (train_percent))

In [ ]:
train_ind

544560

In [ ]:
train_data = encoded_text[:train_ind]
val_data = encoded_text[train_ind:]

### Train the model

In [ ]:
import time
start_time = time.time()

# Variables
epochs = 50
batch_size = 128

seq_len = 100
tracker = 0

num_char = max(encoded_text) + 1

model.train()

if model.use_gpu:
    model.cuda()


for i in range(epochs):
    hidden = model.hidden_state(batch_size)

    for x, y in generate_batches(train_data, batch_size, seq_len):
        tracker += 1

        x = one_hot_encoder(x, num_char)
        inputs = torch.from_numpy(x)
        targets = torch.from_numpy(y)

        if model.use_gpu:
            inputs = inputs.cuda()
            targets = targets.cuda()

        hidden = tuple([state.data for state in hidden])
        model.zero_grad()
        lstm_out, hidden = model.forward(inputs, hidden)
        loss = criterion(lstm_out,targets.view(batch_size*seq_len).long())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm = 5)
        optimizer.step()

        if tracker % 25 == 0:
            val_hidden = model.hidden_state(batch_size = batch_size)
            val_losses = []
            model.eval()

            for x, y in generate_batches(val_data, batch_size, seq_len):
                x = one_hot_encoder(x, num_char)
                inputs = torch.from_numpy(x)
                targets = torch.from_numpy(y)

                if model.use_gpu:
                    inputs = inputs.cuda()
                    targets = targets.cuda()

                val_hidden = tuple([state.data for state in hidden])
                lstm_out, hidden = model.forward(inputs, val_hidden)
                val_loss = criterion(lstm_out, targets.view(batch_size*seq_len).long())

                val_losses.append(val_loss.item())

            model.train()
            print(f"Epoch: {i} Step: {tracker} VAL LOSS: {val_loss.item()}")

final_time = time.time() - start_time
print(f"The training took {final_time/60} minutes")

# save of colab
if platform.system() == 'Linux':
  torch.save(model.state_dict(), '/content/drive/MyDrive/deep learning/courses/pytorch_course/my files/nlp/hidden512_layers3_shakes.pt')
else:
  torch.save(model.state_dict(), 'hidden512_layers3_shakes.pt')

Epoch: 0 Step: 25 VAL LOSS: 3.246267795562744
Epoch: 1 Step: 50 VAL LOSS: 3.2347781658172607
Epoch: 1 Step: 75 VAL LOSS: 3.2337541580200195
Epoch: 2 Step: 100 VAL LOSS: 3.1972055435180664
Epoch: 2 Step: 125 VAL LOSS: 3.0936858654022217
Epoch: 3 Step: 150 VAL LOSS: 3.0141286849975586
Epoch: 4 Step: 175 VAL LOSS: 2.928138494491577
Epoch: 4 Step: 200 VAL LOSS: 2.8028173446655273
Epoch: 5 Step: 225 VAL LOSS: 2.746194839477539
Epoch: 5 Step: 250 VAL LOSS: 2.6873528957366943
Epoch: 6 Step: 275 VAL LOSS: 2.6004631519317627
Epoch: 7 Step: 300 VAL LOSS: 2.538698196411133
Epoch: 7 Step: 325 VAL LOSS: 2.4879000186920166
Epoch: 8 Step: 350 VAL LOSS: 2.4317188262939453
Epoch: 8 Step: 375 VAL LOSS: 2.3599636554718018
Epoch: 9 Step: 400 VAL LOSS: 2.3107879161834717
Epoch: 10 Step: 425 VAL LOSS: 2.2803025245666504
Epoch: 10 Step: 450 VAL LOSS: 2.2527055740356445
Epoch: 11 Step: 475 VAL LOSS: 2.223667860031128
Epoch: 11 Step: 500 VAL LOSS: 2.2003862857818604
Epoch: 12 Step: 525 VAL LOSS: 2.179187774658

In [58]:
model = model = CharModel(
    all_chars=all_characters,
    num_hidden=312,
    num_layers=3,
    drop_prob=0.5,
    use_gpu=use_gpu,
)

# load the model
if platform.system() == 'Linux':
  model.load_state_dict(torch.load('/content/drive/MyDrive/deep learning/courses/pytorch_course/my files/nlp/hidden512_layers3_shakes.pt'))
else:
  model.load_state_dict(torch.load('hidden512_layers3_shakes.pt', map_location=torch.device('cpu')))


### Generate data using the model

We will use 2 functions 1 that given 1 character predict other and other that uses the first one the generate all the text


The topk function basically consists in give the number of max values that you pass as k argument

https://pytorch.org/docs/stable/generated/torch.topk.html

The squeeze function deletes the leftovers axes of the array

https://numpy.org/doc/stable/reference/generated/numpy.squeeze.html

In [ ]:
# Example

x = torch.arange(1., 6.)
x
print(torch.topk(x, 2))

a = np.array([[[1]]]).squeeze()
a


torch.return_types.topk(
values=tensor([5., 4.]),
indices=tensor([4, 3]))


array(1)

In [ ]:
np.random.choice([48, 5, 3], p=[.5, .25, .25])

3

In [ ]:
def predict_next_character(model, char, hidden=None, k=1):
    encoded_text = model.encoder[char]
    encoded_text = np.array([[encoded_text]])
    encoded_text = one_hot_encoder(encoded_text, len(model.all_chars))
    inputs = torch.from_numpy(encoded_text)

    if model.use_gpu:
        inputs = inputs.cuda()

    hidden = tuple([state.data for state in hidden])
    lstm_out, hidden = model.forward(inputs, hidden)
    probs = F.softmax(lstm_out, dim = 1)

    if model.use_gpu:
        probs = probs.cpu()

    probs, index_positions = probs.topk(k)
    index_positions = index_positions.numpy().squeeze()

    probs = probs.detach().numpy().flatten()
    probs = probs/probs.sum()
    char = np.random.choice(index_positions, p=probs)

    return model.decoder[char], hidden

In [ ]:
def generate_text(model, size, seed='The', k=1):
    if model.use_gpu:
        model.cuda()
    else:
        model.cpu()

    model.eval()
    output_chars = [c for c in seed]
    hidden = model.hidden_state(1)

    for char in seed:
        char, hidden = predict_next_character(model, char, hidden, k=k)

    output_chars.append(char)

    for i in range(size):
        char, hidden = predict_next_character(model, output_chars[-1], hidden, k=k)
        output_chars.append(char)

    return ''.join(output_chars)

In [59]:
print(generate_text(model, 1000, 'The ', k=3))

The strong thee
    And the seening that the world thou he the sheet of the
    traest asself, the serpereds of the worthing a marder,
    That we seak make my lord of me to hear
    The world to he the wanter. I will too make
    Well the will and me that too his service,
    The serfort to this thou and make the with a fore thee
    stall to thy fore offing of thine, they have thing and shall see
    and the stares of thee anther and the seem of me.
    To the serves of the streansters.
  CLEOPATRA. That I say you, to madam, the praise of his
    strome that his statt of times the with him a serves,
    And the streeger of this wisting of mine ever of
    to marrer to the store the with to this so that to my.
  PAROLLES. Where is the selfed store the wite.
  ANTONY. I wall see the will the with a far thee are
    she he see her sto than the seems of this store.


                     23
  And the show all that the present the pountess,
  What where to the parting our see the stand,
 